# Module 8: 16 - Multi-Agent Group Chat with AutoGen
----------------------------------------------------------------
In this lesson, we will explore the use of [AutoGen](https://microsoft.github.io/autogen/docs/tutorial/introduction) to create and manage a multi-agent group chat conversation pattern. AutoGen is an open-source framework that leverages multiple agents to enable complex workflows.

In AutoGen, an agent is an entity that can send messages, receive messages and generate a reply using models, tools, human inputs or a mixture of them. This abstraction not only allows agents to model real-world and abstract entities, such as people and algorithms, but it also simplifies implementation of complex workflows as collaboration among agents.

AutoGen is extensible and composable: you can extend a simple agent with customizable components and create workflows that can combine these agents and power a more sophisticated agent, resulting in implementations that are modular and easy to maintain.

## Objectives
* Understand the structure and capabilities of AutoGen for creating multi-agent systems.
* Implement and configure agents using AutoGen.
* Define tools with hardcoded output for testing and experimentation.
* Observe and analyze the collaboration between agents to solve problems or specific challenges.

## What this session covers:
* Introduction to AutoGen and its role in building multi-agent systems.
* Setting up agents and enabling their communication via a message queue.
* Defining and testing tools with hardcoded outputs to facilitate agent interaction.
* Utilizing the Group Chat Manager to manage tasks and orchestrate agent activities.
* Demonstrating how agents collaborate to solve a problem or handle a specific challenge through coordinated task execution.

## References
* https://microsoft.github.io/autogen/docs/tutorial/tool-use/

## Install Libraries

In [ ]:
#!pip3 install pyautogen

## Import Initial Modules

In [1]:
import logging

logging.basicConfig(level=logging.DEBUG)

## Define API KEY

In [2]:
import os
# Set your OpenAI API key as an environment variable
os.environ["OPENAI_API_KEY"] = ""

## Define Tools

#### SOC Analyst

In [3]:
def get_alert(id: str) -> dict:
    """Returns alert information."""
    alerts = {
        "f96a7f37-e7f1-410f-85d3-93056e6601a5": {
            "alert": "Suspicious Login Attempt Detected",
            "severity": "High",
            "indicators": {
                "failed_login_attempts": 5,
                "successful_login_time": "2024-08-05T10:15:30Z",
                "unfamiliar_ip": "192.0.2.123",
                "geo_location": "Unknown Region",
                "affected_user": "jdoe@example.com"
            }
        },
        "a88c5e2a-2a4b-4848-b6e2-8f726a098b4e": {
            "alert": "Malware Detected in Cloud Storage",
            "severity": "Critical",
            "indicators": {
                "file_name": "invoice2024.exe",
                "file_hash": "d41d8cd98f00b204e9800998ecf8427e",
                "detection_time": "2024-08-05T12:45:00Z",
                "quarantine_status": "Quarantined",
                "malware_family": "RansomwareX",
                "affected_users": ["asmith@example.com", "bjones@example.com"]
            }
        },
        "d1b65937-0d8d-4b7e-8d2f-9e3412e0e1a4": {
            "alert": "Unusual Data Transfer Activity",
            "severity": "Medium",
            "indicators": {
                "data_volume": "150GB",
                "transfer_start_time": "2024-08-04T08:00:00Z",
                "transfer_end_time": "2024-08-04T09:30:00Z",
                "external_ip": "203.0.113.45",
                "source_database": "CustomerDB",
                "initiating_user": "jroe@example.com"
            }
        }
    }
    return alerts.get(id, "Alert ID not found")

def analyze_user_behavior(user: str) -> dict:
    """Analyzes the behavior of a user account."""
    # Fake behavior analysis
    behavior_analysis = {
        "user": user,
        "login_history": [
            {"time": "2024-08-01T09:00:00Z", "ip": "198.51.100.23"},
            {"time": "2024-08-02T11:00:00Z", "ip": "198.51.100.23"},
            {"time": "2024-08-05T10:15:30Z", "ip": "192.0.2.123"}
        ],
        "anomalous_activities": ["Suspicious Login Attempt"]
    }
    return behavior_analysis

def analyze_data_transfer(data_volume: str, external_ip: str) -> dict:
    """Analyzes unusual data transfer activities."""
    # Fake data transfer analysis result
    data_transfer_analysis = {
        "data_volume": data_volume,
        "external_ip": external_ip,
        "suspicious": True,
        "potential_exfiltration": True
    }
    return data_transfer_analysis

#### Threat Intelligence Analyst

In [4]:
def analyze_ip(ip: str) -> dict:
    """Enriches information and supports analysis of ip address for reputation and geolocation."""
    # Fake analysis result
    analysis_result = {
        "ip": ip,
        "reputation": "suspicious",
        "geo_location": "Unknown Region",
        "known_associations": ["malware distribution", "botnet activity"]
    }
    return analysis_result

#### Reverse Engineer Analyst

In [5]:
def analyze_file(file_hash: str) -> dict:
    """Analyzes a file for malware signatures and behavior."""
    # Fake file analysis result
    file_analysis_result = {
        "file_hash": file_hash,
        "detected_malware": "RansomwareX",
        "behavior": ["encrypts files", "contacts C2 server"]
    }
    return file_analysis_result

## Config LLM

#### Aggregate Tools

In [6]:
tools_list = [
    {
        "name": "get_alert",
        "description": "Get security context based on alert id",
        "parameters": {
            "type": "object",
            "properties": {
                        "id": {
                            "type": "string",
                            "description": "Id that uniquely identifies an alert",
                        }
                },
            "required": ["id"]}
    },
    {
        "name": "analyze_user_behavior",
        "description": "Analyzes the logging behavior of a user",
        "parameters": {
            "type": "object",
            "properties": {
                        "user": {
                            "type": "string",
                            "description": "Name that uniquely identifies an user",
                        }
                },
            "required": ["user"]}
    },
    {
        "name": "analyze_data_transfer",
        "description": "Analyzes the logging behavior of a user",
        "parameters": {
            "type": "object",
            "properties": {
                        "data_volume": {
                            "type": "string",
                            "description": "Volume of data transferred",
                        },
                        "external_ip": {
                            "type": "string",
                            "description": "Target ip data was transferred to"
                        }
                },
            "required": ["data_volume","external_ip"]}
    },
    {
        "name": "analyze_ip",
        "description": "Enriches information and supports analysis of ip address for reputation and geolocation.",
        "parameters": {
            "type": "object",
            "properties": {
                        "ip": {
                            "type": "string",
                            "description": "IPv4 logical address",
                        }
                },
            "required": ["ip"]}
    },
    {
        "name": "analyze_file",
        "description": "Analyzes a file for malware signatures and behavior",
        "parameters": {
            "type": "object",
            "properties": {
                        "file_hash": {
                            "type": "string",
                            "description": "Hash of the file",
                        }
                },
            "required": ["file_hash"]}
    }
]

#### LLM Config

In [7]:
termination_msg = lambda x: isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

llm_config = { 
    "functions": tools_list,
    "model": "gpt-4-turbo"
}


## Define Agents

#### SOC Analyst Agent

In [8]:
import autogen

soc_analyst_agent = autogen.AssistantAgent(
    name="Soc_Analyst",
    is_termination_msg=termination_msg,
    system_message="""
    You are a security analyst that performs alerts investigations.
    """,
    llm_config=llm_config,
)

soc_analyst_agent.register_function(
    function_map={
        "get_alert": get_alert,
        "analyze_user_behavior": analyze_user_behavior,
        "analyze_data_transfer": analyze_data_transfer,
    }
)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/certifi/cacert.pem'


[autogen.oai.client: 08-06 06:58:02] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


#### Define TI Agent

In [9]:
import autogen

ti_analyst_agent = autogen.AssistantAgent(
    name="Ti_Analyst",
    is_termination_msg=termination_msg,
    system_message="""
    You are an intelligence analyst that enrichs context of alerts investigations.
    """,
    llm_config=llm_config,
)

ti_analyst_agent.register_function(
    function_map={
        "analyze_ip": analyze_ip
    }
)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/certifi/cacert.pem'


[autogen.oai.client: 08-06 06:58:09] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


#### Define Reverse Engineer Agent

In [10]:
import autogen

re_analyst_agent = autogen.AssistantAgent(
    name="Re_Analyst",
    is_termination_msg=termination_msg,
    system_message="""
    You are an reverse engineer analyst that analyzes the content of a file.
    """,
    llm_config=llm_config,
)

re_analyst_agent.register_function(
    function_map={
        "analyze_file": analyze_file
    }
)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/certifi/cacert.pem'


[autogen.oai.client: 08-06 06:58:12] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


#### Define User-Proxy Agent

In [11]:
import autogen

threat_research_agent = autogen.UserProxyAgent(
    name="threat_research_agent",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    system_message="The system who ask questions and give tasks.",
    code_execution_config=False, 
    default_auto_reply="Reply `TERMINATE` if the task is done.",
)

## Define Group Chat

In [12]:
groupchat = autogen.GroupChat(agents=[threat_research_agent, soc_analyst_agent, ti_analyst_agent, re_analyst_agent], messages=[], max_round=5, speaker_selection_method="auto", allow_repeat_speaker=False)
manager = autogen.GroupChatManager(groupchat=groupchat)

## Start Multi-Agent Group Chat

In [13]:
threat_research_agent.initiate_chat(
    manager,
    message= "Investigate alert f96a7f37-e7f1-410f-85d3-93056e6601a5 and provide a summary about it with all details"
)

threat_research_agent (to chat_manager):

Investigate alert f96a7f37-e7f1-410f-85d3-93056e6601a5 and provide a summary about it with all details

--------------------------------------------------------------------------------

Next speaker: Soc_Analyst



DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': '\n    You are a security analyst that performs alerts investigations.\n    ', 'role': 'system'}, {'content': 'Investigate alert f96a7f37-e7f1-410f-85d3-93056e6601a5 and provide a summary about it with all details', 'name': 'threat_research_agent', 'role': 'user'}], 'model': 'gpt-4-turbo', 'functions': [{'name': 'get_alert', 'description': 'Get security context based on alert id', 'parameters': {'type': 'object', 'properties': {'id': {'type': 'string', 'description': 'Id that uniquely identifies an alert'}}, 'required': ['id']}}, {'name': 'analyze_user_behavior', 'description': 'Analyzes the logging behavior of a user', 'parameters': {'type': 'object', 'properties': {'user': {'type': 'string', 'description': 'Name that uniquely identifies an user'}}, 'required': ['user']}}, {'name': 'analyze_data_transfer', 'description': 'Analyzes the logging 

Soc_Analyst (to chat_manager):

***** Suggested function call: get_alert *****
Arguments: 
{"id":"f96a7f37-e7f1-410f-85d3-93056e6601a5"}
**********************************************

--------------------------------------------------------------------------------

Next speaker: Soc_Analyst


>>>>>>>> EXECUTING FUNCTION get_alert...
Soc_Analyst (to chat_manager):

***** Response from calling function (get_alert) *****
{'alert': 'Suspicious Login Attempt Detected', 'severity': 'High', 'indicators': {'failed_login_attempts': 5, 'successful_login_time': '2024-08-05T10:15:30Z', 'unfamiliar_ip': '192.0.2.123', 'geo_location': 'Unknown Region', 'affected_user': 'jdoe@example.com'}}
******************************************************

--------------------------------------------------------------------------------

Next speaker: Ti_Analyst



DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': '\n    You are an intelligence analyst that enrichs context of alerts investigations.\n    ', 'role': 'system'}, {'content': 'Investigate alert f96a7f37-e7f1-410f-85d3-93056e6601a5 and provide a summary about it with all details', 'name': 'threat_research_agent', 'role': 'user'}, {'content': '', 'function_call': {'arguments': '{"id":"f96a7f37-e7f1-410f-85d3-93056e6601a5"}', 'name': 'get_alert'}, 'name': 'Soc_Analyst', 'role': 'assistant'}, {'content': "{'alert': 'Suspicious Login Attempt Detected', 'severity': 'High', 'indicators': {'failed_login_attempts': 5, 'successful_login_time': '2024-08-05T10:15:30Z', 'unfamiliar_ip': '192.0.2.123', 'geo_location': 'Unknown Region', 'affected_user': 'jdoe@example.com'}}", 'name': 'get_alert', 'role': 'function'}], 'model': 'gpt-4-turbo', 'functions': [{'name': 'get_alert', 'description': 'Get security co

Ti_Analyst (to chat_manager):

### Alert Summary

**Alert ID:** f96a7f37-e7f1-410f-85d3-93056e6601a5  
**Description:** Suspicious Login Attempt Detected  
**Severity:** High

### Indicators:
- **Failed Login Attempts:** 5
- **Successful Login Time:** 2024-08-05 10:15:30 (UTC)
- **IP Address:** 192.0.2.123 (Geo Location: Unknown Region)
- **Affected User:** jdoe@example.com 

### Initial Analysis:
The alert indicates a high severity situation involving suspicious login attempts. There were 5 failed login attempts before a successful access was granted. The successful login was from an IP address not previously associated with the user, pointing to a potential security breach.

To provide additional context and a deeper analysis, I will proceed to investigate the unfamiliar IP and check the recent behavior of the affected user.
***** Suggested function call: analyze_ip *****
Arguments: 
{"ip":"192.0.2.123"}
***********************************************

-------------------------------

ChatResult(chat_id=None, chat_history=[{'content': 'Investigate alert f96a7f37-e7f1-410f-85d3-93056e6601a5 and provide a summary about it with all details', 'role': 'assistant'}, {'content': '', 'function_call': {'arguments': '{"id":"f96a7f37-e7f1-410f-85d3-93056e6601a5"}', 'name': 'get_alert'}, 'name': 'Soc_Analyst', 'role': 'assistant'}, {'content': "{'alert': 'Suspicious Login Attempt Detected', 'severity': 'High', 'indicators': {'failed_login_attempts': 5, 'successful_login_time': '2024-08-05T10:15:30Z', 'unfamiliar_ip': '192.0.2.123', 'geo_location': 'Unknown Region', 'affected_user': 'jdoe@example.com'}}", 'name': 'get_alert', 'role': 'function'}, {'content': '### Alert Summary\n\n**Alert ID:** f96a7f37-e7f1-410f-85d3-93056e6601a5  \n**Description:** Suspicious Login Attempt Detected  \n**Severity:** High\n\n### Indicators:\n- **Failed Login Attempts:** 5\n- **Successful Login Time:** 2024-08-05 10:15:30 (UTC)\n- **IP Address:** 192.0.2.123 (Geo Location: Unknown Region)\n- **Aff